# PACKAGES AND LIBRARIES

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from warnings import filterwarnings
from mpl_toolkits.mplot3d import Axes3D
import statsmodels.api as sm
import missingno as msno
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.neighbors import LocalOutlierFactor
from scipy.stats import levene
from scipy.stats import shapiro
from scipy.stats.stats import pearsonr
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.preprocessing import scale
from sklearn.model_selection import ShuffleSplit, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import model_selection
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
import xgboost as xgb
from xgboost import XGBRegressor, XGBClassifier
from lightgbm import LGBMRegressor, LGBMClassifier
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import tree
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, roc_curve

#### WARNINGS 

In [ ]:
filterwarnings("ignore", category=DeprecationWarning) 
filterwarnings("ignore", category=FutureWarning) 
filterwarnings("ignore", category=UserWarning)

# DATA

In [ ]:
DigitsTrain = pd.read_csv("../input/digit-recognizer/train.csv") # train main

In [ ]:
DigitsTest = pd.read_csv("../input/digit-recognizer/test.csv") # test main

In [ ]:
data = DigitsTrain.copy() # to protect train main data

In [ ]:
dataTest = DigitsTest.copy() # to protect test main data

# EXPLORATORY DATA ANALYSIS

#### TRAIN DATA

In [ ]:
print(data.head())

In [ ]:
print(data.shape)

In [ ]:
print(data.columns)

In [ ]:
print(data.columns.value_counts().sum())

In [ ]:
print(data.info())

In [ ]:
print(data.describe().T)

In [ ]:
print(data.index)

In [ ]:
print(type(data)) # checking type

In [ ]:
print(data.corr()) # checking correlation
# it is not suitable for this data, but you should use and try for others

In [ ]:
print(data.cov()) # checking covariance
# it is not suitable for this data, but you should use and try for others

In [ ]:
print(data.groupby(["label"])["pixel0"].mean()) # example for groupby
# it is not suitable for this data, but you should use and try for others

In [ ]:
print(data.groupby(["label"])["pixel783"].mean()) # example for groupby
# it is not suitable for this data, but you should use and try for others

In [ ]:
maxPixel783 = data[data["pixel783"] == data["pixel783"].max()] # example for checking max
# it is not suitable for this data, but you should use and try for others
print(maxPixel783.index.value_counts())

In [ ]:
print(data.isnull().all().sum()) # checking all missing values for rows and columns

In [ ]:
print(data.isnull().any().sum()) # checking any missing values for rows and columns

In [ ]:
print(data.duplicated().sum()) # checking duplicated

#### TEST DATA

In [ ]:
print(dataTest.head())

In [ ]:
print(dataTest.shape)

In [ ]:
print(dataTest.columns)

In [ ]:
print(dataTest.info())

In [ ]:
print(dataTest.describe().T)

In [ ]:
print(dataTest.corr())

In [ ]:
print(dataTest.cov())

In [ ]:
print(dataTest.columns.value_counts().sum())

In [ ]:
print(dataTest.isnull().all().sum())

In [ ]:
print(dataTest.isnull().any().sum())

In [ ]:
print(dataTest.duplicated().sum())

# SPLITTING TRAIN AND TEST FOR TRAINING

In [ ]:
x = data.drop("label",axis=1) # features
y = data["label"] # prediction target

In [ ]:
xTrain,xTest,yTrain,yTest = train_test_split(x,y,test_size=0.2,random_state=42) # for testing our models

In [ ]:
scaler = MinMaxScaler()

In [ ]:
xTrain = scaler.fit_transform(xTrain)
xTest = scaler.fit_transform(xTest)
# for faster process
# you can also use StandartScaler

In [ ]:
print(xTrain.shape)

In [ ]:
print(xTest.shape)

# VISUALIZATION

In [ ]:
figure = plt.figure(figsize=(15,8))
sns.countplot(data["label"],color="black")
# we see how many numbers are in the data
plt.show()

In [ ]:
for i in range(0,5):
    image = x.iloc[i]
    image = np.array(image,dtype='uint8')
    # we need to transform it to numpy array for visualizing
    image = image.reshape((28,28))
    # we need to reshape for visualizing because of pixels
    plt.imshow(image,cmap="gray")
    plt.axis("off")
    plt.show()
# visualizing numbers in data by using pixel values

# MODELS

#### CLASSIFIER

In [ ]:
# models training
lj = LogisticRegression(solver="liblinear").fit(xTrain,yTrain)
gnb = GaussianNB().fit(xTrain,yTrain)
knnc = KNeighborsClassifier().fit(xTrain,yTrain)
cartc = DecisionTreeClassifier(random_state=42).fit(xTrain,yTrain)
rfc = RandomForestClassifier(random_state=42,verbose=False).fit(xTrain,yTrain)

In [ ]:
modelsc = [lj,gnb,knnc,cartc,rfc]
# we will try 5 models for example
# if you want you can try all of them

In [ ]:
# models' accuracy scores
for model in modelsc:
    name = model.__class__.__name__
    predict = model.predict(xTest)
    R2CV = cross_val_score(model,xTest,yTest,cv=10,verbose=False).mean()
    error = -cross_val_score(model,xTest,yTest,cv=10,scoring="neg_mean_squared_error",verbose=False).mean()
    print(name + ": ")
    print("-" * 10)
    print("ACC-->",accuracy_score(yTest,predict))
    print("R2CV-->",R2CV)
    print("MEAN SQUARED ERROR-->",np.sqrt(error))
    print("-" * 30)

as we see, the best is Random Forest for each of 5 models

#### PREDICTION

In [ ]:
testlabel = pd.read_csv("../input/digit-recognizer/test.csv")

 we use "dataTest" for ANN, that is why we need to define again

In [ ]:
predictC = rfc.predict(testlabel)

In [ ]:
print(predictC[0])

In [ ]:
# checking same value
imageT = testlabel.iloc[0]
imageT = np.array(imageT,dtype='uint8')
# we need to transform it to numpy array for visualizing
imageT = imageT.reshape((28,28))
# we need to reshape for visualizing because of pixels
plt.imshow(imageT,cmap="gray")
plt.axis("off")
plt.show()

IT'S TRUE

#### SUBMISSION

In [ ]:
print(predictC.shape)

In [ ]:
imageIDC = [h+1 for h in range(len(predictClass))]

In [ ]:
print(imageIDC)

In [ ]:
submissionClass = pd.DataFrame()
submissionClass["ImageId"] = imageIDC
submissionClass["Label"] = predictC

In [ ]:
submissionC.to_csv('submissionClass.csv', index=False)

# ANN MODEL

#### LIBRARY 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
import tensorflow as tf

#### TRAIN

In [ ]:
xN = data.drop("label",axis=1)
yN = data["label"]

In [ ]:
xN = np.array(xN)
xN = xN.reshape(42000, 28, 28)

In [ ]:
print(xN.shape)

#### CREATING

In [ ]:
ANNmodel = tf.keras.models.Sequential([
  # inputs
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  # hiddens
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  # output
  tf.keras.layers.Dense(10)
])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
ANNmodel.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

MainModel = ANNmodel.fit(xN, yN, validation_split=0.2, epochs=30, callbacks=[callback])

In [ ]:
print(ANNmodel.summary())

In [ ]:
loss = pd.DataFrame(ANNmodel.history.history)

In [ ]:
print(loss.head())

In [ ]:
loss.plot()

In [ ]:
plt.plot(MainModel.history["accuracy"])
plt.plot(MainModel.history["val_accuracy"])
plt.xlabel("EPOCH")
plt.ylabel("ACC")
plt.legend()
plt.show()

#### PREDICTION

In [ ]:
dataTest = np.array(dataTest)

In [ ]:
print(dataTest.shape)

In [ ]:
dataTest = dataTest.reshape(28000,28,28)

In [ ]:
predictANN = ANNmodel.predict(dataTest)

In [ ]:
plt.imshow(dataTest[0])
plt.show()

In [ ]:
p = [np.argmax(i) for i in predictANN]

In [ ]:
# checking same value
print(p[0])

IT'S TRUE

#### SUBMISSION

In [ ]:
imageID = [x+1 for x in range(len(p))]

In [ ]:
submission = pd.DataFrame()
submission["ImageId"] = imageID
submission["Label"] = p

In [ ]:
submission.to_csv('submissionANN.csv', index=False)